### Import Libraries

In [12]:
import pandas as pd
import sqlite3

### Creating the database

In [13]:
def create_database():
    try:
        conn = sqlite3.connect('account.db')
        cur = conn.cursor()
        return cur, conn
    except Exception as e:
        print(f"Error creating or connecting to database: {e}")
        return None, None

### Executing the database

In [14]:
def execute_queries(cur, conn, queries):
    try:
        for query in queries:
            cur.execute(query)
        conn.commit()
    except Exception as e:
        print(f"Error executing queries: {e}")

### Inserting the data

In [15]:
def insert_data(cur, conn, table_name, dataframe):
    try:
        dataframe.to_sql(table_name, conn, if_exists='replace', index=False)
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

### Creating the tables and connecting to the account database

In [16]:
def main():
    # Create and connect to the 'account' database
    cur, conn = create_database()

    if cur is None or conn is None:
        print("Failed to create or connect to the database.")
        return

    # Load datasets
    try:
        accountscountry = pd.read_csv(
            'Desktop/#From Eghosa/DE projects/Wealth Data/Wealth-AccountsCountry.csv', 
            usecols=["Code", "Short Name", "Table Name", "Long Name", "Currency Unit"]
        ).rename(columns={
            "Code": "code",
            "Short Name": "short_name",
            "Table Name": "table_name",
            "Long Name": "long_name",
            "Currency Unit": "currency_unit"
        })

        accountsdata = pd.read_csv(
            'Desktop/#From Eghosa/DE projects/Wealth Data/Wealth-AccountData.csv',
            usecols=["Country Name", "Country Code", "Series Name", "Series Code",
                     "2000 [YR2000]", "2005 [YR2005]", "2010 [YR2010]", "2015 [YR2015]", "2018 [YR2018]"]
        ).rename(columns={
            'Country Name': 'country_name',
            'Country Code': 'country_code',
            'Series Name': 'indicator_name',
            'Series Code': 'indicator_code',
            '2000 [YR2000]': 'year_2000', 
            '2005 [YR2005]': 'year_2005', 
            '2010 [YR2010]': 'year_2010', 
            '2015 [YR2015]': 'year_2015', 
            '2018 [YR2018]': 'year_2018'
        })

        accountseries = pd.read_csv(
            'Desktop/#From Eghosa/DE projects/Wealth Data/Wealth-AccountSeries.csv',
            usecols=["Code", "Topic", "Indicator Name", "Long definition", "Source"]
        ).rename(columns={
            'Code': 'series_code',
            'Topic': 'topic',
            'Indicator Name': 'indicator_name',
            'Long definition': 'long_definition',
            'Source': 'source'
        })
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return

    # SQL queries to create tables
    create_table_queries = [
        """CREATE TABLE IF NOT EXISTS accountscountry(
            code VARCHAR PRIMARY KEY,
            short_name VARCHAR,
            table_name VARCHAR,
            long_name VARCHAR,
            currency_unit VARCHAR
        )""",
        """CREATE TABLE IF NOT EXISTS accountsdata(
            country_name VARCHAR,
            country_code VARCHAR,
            indicator_name VARCHAR,
            indicator_code VARCHAR,
            year_2000 VARCHAR,
            year_2005 VARCHAR,
            year_2010 VARCHAR,
            year_2015 VARCHAR,
            year_2018 VARCHAR
        )""",
        """CREATE TABLE IF NOT EXISTS accountseries(
            series_code VARCHAR PRIMARY KEY,
            topic VARCHAR,
            indicator_name VARCHAR,
            long_definition TEXT,
            source VARCHAR
        )"""
    ]

    # Execute table creation queries
    execute_queries(cur, conn, create_table_queries)

    # Insert data into tables
    insert_data(cur, conn, "accountscountry", accountscountry)
    insert_data(cur, conn, "accountsdata", accountsdata)
    insert_data(cur, conn, "accountseries", accountseries)

    # Close connection
    cur.close()
    conn.close()

if __name__ == "__main__":
    main()


In [17]:
main()

### lets use the ipython-sql library to inspect a table

In [ ]:
!pip install ipython-sql

In [19]:
%load_ext sql
%sql sqlite:///account.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [20]:
%sql SELECT * FROM accountscountry limit 3


 * sqlite:///account.db
Done.


code,long_name,currency_unit,table_name,short_name
ALB,Republic of Albania,Albanian lek,Albania,Albania
ARG,Argentine Republic,Argentine peso,Argentina,Argentina
ARM,Republic of Armenia,Armenian dram,Armenia,Armenia


In [21]:
%sql SELECT * FROM accountseries limit 3


 * sqlite:///account.db
Done.


series_code,indicator_name,long_definition,source,topic
NW.HCA.TO,Human capital (constant 2018 US$),"Human capital is computed as the present value of future earnings for the working population over their lifetimes. Values are measured at market exchange rates in constant 2018 US dollars, using a country-specific GDP deflator.","World Bank. 2021. The Changing Wealth of Nations 2021: Managing Assets for the Future. Washington, DC: World Bank. doi:10.1596/978-1-4648-1590-4.",Human capital
NW.HCA.PC,Human capital per capita (constant 2018 US$),"Human capital is computed as the present value of future earnings for the working population over their lifetimes. Values are measured at market exchange rates in constant 2018 US dollars, using a country-specific GDP deflator.","World Bank. 2021. The Changing Wealth of Nations 2021: Managing Assets for the Future. Washington, DC: World Bank. doi:10.1596/978-1-4648-1590-4.",Human capital
NW.HCA.FEMP.PC,"Human capital per capita, employed female (constant 2018 US$)","Human capital is computed as the present value of future earnings for the working population over their lifetimes. Values are measured at market exchange rates in constant 2018 US dollars, using a country-specific GDP deflator.","World Bank. 2021. The Changing Wealth of Nations 2021: Managing Assets for the Future. Washington, DC: World Bank. doi:10.1596/978-1-4648-1590-4.",Human capital


In [22]:
%sql SELECT * FROM accountsdata limit 3


 * sqlite:///account.db
Done.


country_name,country_code,indicator_name,indicator_code,year_2000,year_2005,year_2010,year_2015,year_2018
Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.36E+10,54400000000.0,68100000000.0,73700000000.0,8.12E+10
Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14118.13,18072.3,23362.86,25593.71,28333.68
Albania,ALB,"Human capital per capita, employed female (constant 2018 US$)",NW.HCA.FEMP.PC,3424.007,2222.115,2889.15,2987.545,3312.927
